In [ ]:
!pip install ultralytics --quiet

from ultralytics import YOLO
from pathlib import Path
import torch

PLATFORM = "kaggle"
EXPORT_FORMAT = "onnx"

BASE_PATH = Path("/kaggle/input/food2022").resolve()
YAML_PATH = BASE_PATH / "food2022.yaml"
MODEL_NAME = "yolov8n.pt"
RUN_NAME = f"food2022_yolov8_{PLATFORM}"

EPOCHS = 10
BATCH = 32
IMGSZ = 640
WORKERS = 2

def get_device():
    if torch.cuda.is_available():
        return "cuda"
    else:
        return "cpu"

def train():
    assert YAML_PATH.exists(), f"YAML file not found: {YAML_PATH}"
    device = get_device()
    print(f"Platform: {PLATFORM.upper()} | Device: {device.upper()}")
    model = YOLO(MODEL_NAME)
    model.train(
        data=str(YAML_PATH),
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        workers=WORKERS,
        device=device,
        amp=False,
        cache=True,
        rect=True,
        name=RUN_NAME
    )
    if EXPORT_FORMAT:
        model.export(format=EXPORT_FORMAT)
        from shutil import copyfile
        copyfile("yolov8n.onnx", "/kaggle/working/yolov8n.onnx")

if __name__ == "__main__":
    train()